# Clustering Neighborhoods in Toronto

## Wikipedia Scraping

Import Libraries

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np

Parse the Wikipedia data

In [2]:
url = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup = BeautifulSoup(url,'lxml')
table = soup.find('table',{'class':'wikitable sortable'})
table_rows = table.findAll('tr')

Iterate through rows and append them to list

In [3]:
list =[]
for tr in table_rows:
    td = tr.findAll('td')
    row = [tr.text for tr in td]
    list.append(row)

Create Pandas dataframe
The dataframe will consist of three columns: PostalCode, Borough, and Neighborhood

In [4]:
df = pd.DataFrame(list, columns=["Postcode","Borough","Neighbourhood"])
df.replace('\n','',regex=True, inplace=True)

Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.

In [6]:
df.dropna(axis=0, inplace=True)
not_assigned = df[df['Borough'] == 'Not assigned'].index
df.drop(not_assigned, inplace = True)

If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough. So for the 9th cell in the table on the Wikipedia page, the value of the Borough and the Neighborhood columns will be Queen's Park.

In [7]:
df.loc[df.Neighbourhood == 'Not assigned', 'Neighbourhood'] = df.Borough

More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, you will notice that M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. These two rows will be combined into one row with the neighborhoods separated with a comma as shown in row 11 in the above table.

In [8]:
df_cleansed = df.groupby(['Postcode','Borough'])['Neighbourhood'].apply(lambda x: ', '.join(x.astype(str))).reset_index()

In the last cell of your notebook, use the .shape method to print the number of rows of your dataframe.

In [9]:
df_cleansed.shape

(103, 3)